In [6]:
import numpy as np
import pandas as pd
import os

In [7]:
data_path = r'C:\Projects\connecteddatahub\data'
leadership_path = os.path.join(data_path, 'cleaned_dataframes', 'master_leadershipdata.csv')

leadership_df = pd.read_csv(leadership_path)
print(leadership_df.columns)

Index(['Year', 'Name', 'Position', 'Institution', 'SubInstitution',
       'Education', 'FixedPosition', 'Seniority', 'Designation',
       'AffiliationId', 'carnegie_id', 'SystemId', 'RawName', 'RawInstitution',
       'RawPosition', 'FirstName', 'MiddleName', 'LastName', 'NickName',
       'SuffixName', 'PrefixName', 'MiddleInitials', 'RawName4Match',
       'LeaderId'],
      dtype='object')


In [8]:
import pandas as pd
import numpy as np

def detect_board_size_anomalies(
    leadership_df: pd.DataFrame,
    position_value: str = "Board Member",
    threshold: float = 0.30
) -> pd.DataFrame:
    """
    Detect years where an institution's board size differs from its own average by >= threshold.
    Includes original row indices for quick lookup.
    """
    df = leadership_df.copy()
    df["_orig_index"] = df.index  # Preserve original index

    # 1) Filter board members only
    fp = df["FixedPosition"].astype(str).str.strip().str.lower()
    df = df[fp.eq(position_value.lower())]

    # 2) Build robust unique identifier
    def _coalesce_member_key(row):
        for col in ("LeaderId", "RawName4Match", "Name"):
            val = row.get(col)
            if pd.notna(val) and str(val).strip() != "":
                return str(val).strip()
        return None
    df["member_key"] = df.apply(_coalesce_member_key, axis=1)

    # 3) Compute board size and track original indices
    board_sizes = (
        df.groupby(["AffiliationId", "Year"], dropna=False)
          .agg(board_size=("member_key", "nunique"),
               orig_indices=("_orig_index", lambda x: list(x)))
          .reset_index()
    )

    # 4) Historical averages per institution
    aff_stats = (
        board_sizes.groupby("AffiliationId", dropna=False)["board_size"]
                   .agg(avg_board_size="mean", years_counted_for_avg="count")
                   .reset_index()
    )

    # 5) Merge and compute deviations
    out = board_sizes.merge(aff_stats, on="AffiliationId", how="left")
    out = out[out["avg_board_size"] > 0]
    out["abs_diff"] = out["board_size"] - out["avg_board_size"]
    out["pct_diff"] = out["abs_diff"] / out["avg_board_size"]

    # 6) Flag anomalies
    anomalies = out[out["pct_diff"].abs() >= threshold].copy()

    # 7) Sort chronologically and tidy
    anomalies = (
        anomalies[[
            "Year", "AffiliationId", "board_size", "avg_board_size",
            "abs_diff", "pct_diff", "years_counted_for_avg", "orig_indices"
        ]]
        .sort_values(["Year", "AffiliationId"])
        .reset_index(drop=True)
    )

    return anomalies


# --- Example usage ---
anomalies = detect_board_size_anomalies(leadership_df, position_value="Board Member", threshold=0.30)

if anomalies.empty:
    print("No ≥30% board-size anomalies detected.")
else:
    print(
        anomalies.to_string(
            index=True,  # show row numbers
            formatters={
                "avg_board_size": "{:.2f}".format,
                "abs_diff": "{:+.2f}".format,
                "pct_diff": lambda x: f"{x:+.1%}"
            }
        )
    )


     Year  AffiliationId  board_size avg_board_size abs_diff pct_diff  years_counted_for_avg                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       orig_indices
0    1999       29957033          12          18.09    -6.09   -33.7%                     11                                                                                                            